# ChatGPT for SQL Querying

Purpose: to write SQL queries for a specific database based on plain-text requests

This program can be used by those without SQL training to query a database. Often, non-analysts rely on those with SQL training to access information. This program can eliminate the "middle man" and allow users to query a database using plain-text. The program essentially translates plain-text to SQL queries. Users can plug the output into a database.

In [19]:
import openai as ai
import pandas as pd

# Set Openai key (key omitted for publication)
ai.api_key = api_key

As with any ChatGPT model, its usefullness comes from the "messages" portion of the function. This is filled with the specifics of the database which the model will query. A more robust model will include details on what each column contains, including data types and colloquial names for the information therein. 

In [20]:
# Create Marlon (a name for the bot)
def Marlon(prompt):
    messages = [
        {"role": "assistant", "content": '''I am a bot meant to help create SQL queries for the user. 
         The user will tell me what they want, but they do not know how to write a SQL query or the exact 
         column names. My only response is the query in proper SQL syntax. I do not include any commentary.'''},
        {"role": "assistant", "content": '''The querying will be performed on a database of scouting 
         reports. The database is comprised of the following tables and columns. Some columns include brief 
         explanations of their content 
         
            player_info:
            Columns:
            player_id
            first_name
            last_name
            pos
            secondary_pos
            bats
            throws
            organization
            level
            b_ref
            date_of_birth
            height_in
            weight_lb
            country
            college_1
            college_2
            high_school
            state
         
            report_info:

            Columns:
            report_id
            player_id
            first_name
            last_name
            scout
            report_date
            report_level
            report_link
         
            report_grades_pitcher:

            Columns:
            report_id
            player_id
            first_name
            last_name
            future_value
            fastball_present - graded on a 20-80 scale based on the scout's opinion of the player's
                                 current fastball
            fastball_future - graded on a 20-80 scale based on the scout's opinion of the player's 
                                fastball's potential
            changeup_present
            changeup_future
            slider_present
            slider_future
            curveball_present
            curveball_future
            cutter_present
            cutter_future
            sinker_present
            sinker_future
            splitter_present
            splitter_future
            sweeper_present
            sweeper_future
            knuckleball_present
            knuckleball_future
            control_present
            control_future
            command_present
            command_future
         
            report_summaries_pitcher:

            Columns:
            report_id
            player_id
            first_name
            last_name
            summary
            tool_summary
            release_point
            arm_action
            delivery
            approach
            sequencing
            defense_movement
            mound_presence
            run_game_control
            release_time
            physical_description
            makeup
            control
            command
            look_comments
            other_comments
            fastball
            fastball_velo_top - maximum velocity of the fastball
            fastball_velo_bottom - minimum velocity of the fastball
            changeup
            changeup_velo_top
            changeup_velo_bottom
            slider
            slider_velo_top
            slider_velo_bottom
            curveball
            curveball_velo_top
            curveball_velo_bottom
            cutter
            cutter_velo_top
            cutter_velo_bottom
            sinker
            sinker_velo_top
            sinker_velo_bottom
            splitter
            splitter_velo_top
            splitter_velo_bottom
            sweeper
            sweeper_velo_top
            sweeper_velo_bottom
            knuckleball
            knuckleball_velo_top
            knuckleball_velo_bottom
         
            report_grades_hitter:

            Columns:
            report_id
            player_id
            first_name
            last_name
            future_value
            hit_present
            hit_future
            power_raw_present
            power_raw_future
            power_production_present
            power_production_future
            offensive_production_present
            offensive_production_future
            fielding_present
            fielding_future
            range_present
            range_future
            field_aggressiveness_present
            field_aggressiveness_future
            hands_present
            hands_future
            arm_strength_present
            arm_strength_future
            arm_accuracy_present
            arm_accuracy_future
            baserunning_present
            baserunning_future
            run_speed_present
            run_speed_future
            receiving_present
            receiving_future
            run_game_control_present
            run_game_control_future
            blocking_present
            blocking_future
            footwork_present
            footwork_future
            framing_present
            framing_future
            arm_present
            arm_future
            c_defense_present
            c_defense_future
         
            report_summaries_hitter:

            Columns:
            report_id
            player_id
            first_name
            last_name
            Summary
            hit_summary
            defense_summary
            base_summary
            hit_approach
            hit_adjustment
            swing_mechanics
            defense_reads_movement
            hands_feet
            receiving
            run_game_control
            base_reads
            base_aggressiveness
            physical_description
            makeup
            h_1st
            steal
            look_comments
            other_comments'''},
        {"role": "user", "content": prompt}
    ]

    response = ai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        max_tokens=200
    )

    answer = response.choices[0].message["content"]
    tokens_used = response["usage"]["total_tokens"]

    result_dict = {"answer": answer, "tokens_used": tokens_used}
    return result_dict

In [15]:
def ask_marlon():
    Input = input("What would you like to ask Marlon?")
    result = Marlon(Input)
    answer = result["answer"]
    return print(answer)



In [16]:
# User input: 'Show me the names of the guys who are over 6 feet tall and throw 90mph or faster.'

ask_marlon()

SELECT first_name, last_name
FROM player_info
WHERE height_in > 72 --6 feet = 72 inches
AND player_id IN (
    SELECT player_id
    FROM report_grades_pitcher
    WHERE fastball_velo_top >= 90
) ;


In [17]:
# User input: 'Show me the names, future values, and summaries of the pitchers with future value 55 or higher.'

ask_marlon()

SELECT first_name, last_name, future_value, summary
FROM report_grades_pitcher
JOIN report_summaries_pitcher USING (report_id, player_id, first_name, last_name)
WHERE future_value >= 55;


In [18]:
# User input: 'Show me the average fastball velocity of pitchers broken down by height.'

ask_marlon()

SELECT height_in, AVG(fastball_velo_top) AS average_fastball_velocity
FROM report_summaries_pitcher
GROUP BY height_in;
